# Credentials and initialize conneciton

Follow https://developers.google.com/earth-engine/reference/Quickstart#before-you-begin to get a secret key

In [27]:
import json

KEY = 'my-secret-key.json'
with open(KEY, 'r') as f:
    data = json.load(f)
    SERVICE_ACCOUNT = data['client_email']
    PROJECT = data['project_id']

In [21]:
import ee

ee_creds = ee.ServiceAccountCredentials(SERVICE_ACCOUNT, KEY)
ee.Initialize(ee_creds)

In [32]:
import geemap

# Bounding box: [west, south, east, north]
bbox = ee.Geometry.BBox(-122.5, 37.0, -121.5, 38.0)

start_date = '2024-06-01'
end_date = '2024-06-30'

# 🆕 Use harmonized Sentinel-2
s2 = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(bbox) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .sort('system:time_start', False)

# Cloud mask using Scene Classification Layer (SCL)
def compute_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    scl = image.select('SCL')
    # Mask: cloud shadow (3), medium/high cloud (8, 9), cirrus (10)
    mask = scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10))
    return image.addBands(ndvi).updateMask(mask)

# Map function and get most recent image with valid pixels
s2_ndvi = s2.map(compute_ndvi)
latest_ndvi = s2_ndvi.select('NDVI').mosaic().clip(bbox)

# NDVI visualization
ndvi_vis = {
    'min': 0.0,
    'max': 1.0,
    'palette': ['white', 'green']
}

# Display with geemap
Map = geemap.Map(center=[37.5, -122.0], zoom=9)
Map.addLayer(latest_ndvi, ndvi_vis, 'Latest NDVI')
Map.addLayer(bbox, {}, 'Bounding Box')
Map


Map(center=[37.5, -122.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…